In [1]:
from transformers import pipeline
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import numpy as np
import os
import collections
import matplotlib.pyplot as plt
from collections import defaultdict

In [2]:
root_path = "/home/oscar/Documents/Workspace/11785/project/penstate_data/download/Full_voice_files"
female_path = root_path + "/ADAPT_Female_voice_recordings"
male_path = root_path + "/ADAPT_Male_voice_recordings"

In [3]:
female_list = os.listdir(female_path)
male_list = os.listdir(male_path)

In [4]:
def get_phoneme_len(predicted_id):
    predicted_id = np.array(predicted_id)
    length = len(predicted_id)
    nonzero = np.nonzero(np.array(predicted_id))[0]

    res_len = []
    res_id = []
    prev_index = 0
    for idx, item in enumerate(nonzero):
        if idx == 0:
            end_index = int((nonzero[1]+nonzero[0])/2)
        elif idx == len(nonzero) - 1:
            res_id.append(predicted_id[item])
            end_index = length
            res_len.append(end_index - prev_index)
        else:
            # if nonzero[idx] - nonzero[idx-1] == 1 and predicted_id[nonzero[idx-1]] != predicted_id[nonzero[idx]]:
            #     print(nonzero[idx-1])
            if nonzero[idx] - nonzero[idx-1] > 1:
                res_id.append(predicted_id[nonzero[idx-1]])
                res_len.append(end_index - prev_index)
                prev_index = end_index
            end_index = int((nonzero[idx] + nonzero[idx + 1]) / 2)

    return res_len, res_id

In [5]:
# load model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

## Female

In [43]:
output_root_path = "/home/oscar/Documents/Workspace/11785/project/penstate_data/extract_phoneme/female/"

In [74]:
for voice_file in female_list:
    print(voice_file)
    voice_absolute_path = female_path + "/" + voice_file
    people_id = voice_file.split(".")[0]
    # print(voice_absolute_path)
    voice, sample_rate = torchaudio.load(voice_absolute_path)
    input_values = processor(voice[0][::2], return_tensors="pt").input_values
    with torch.no_grad():
        logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    # transcription = processor.batch_decode(predicted_ids)
    
    ratio = len(voice[0]) / len(predicted_ids[0])
    
    res_len, res_id = get_phoneme_len(predicted_ids[0])
    
    cum_sum = np.cumsum(res_len)
    enlarged_cum_sum = (cum_sum * ratio).astype(int)
    
    dic = defaultdict(int)
    
    for i in range(len(res_id)):
        if not os.path.exists(output_root_path + str(res_id[i])):
            os.mkdir(output_root_path + str(res_id[i]))
        curr_phoneme_output_path = output_root_path + str(res_id[i]) + "/" + people_id + '_' + str(res_id[i]) + '_' + str(dic[res_id[i]]) + '.npy'
        if i == 0:
            np.save(curr_phoneme_output_path, np.array(voice[0][max(enlarged_cum_sum[i] - 5000, 0):enlarged_cum_sum[i]]))
            # print(max(enlarged_cum_sum[i] - 20, 0), enlarged_cum_sum[i])
        elif i == len(res_id)-1:
            np.save(curr_phoneme_output_path, np.array(voice[0][enlarged_cum_sum[i - 1]:min(enlarged_cum_sum[i], enlarged_cum_sum[i - 1] + 20)]))
            # print(enlarged_cum_sum[i - 1], min(enlarged_cum_sum[i], enlarged_cum_sum[i - 1] + 5000))
        else:
            np.save(curr_phoneme_output_path, np.array(voice[0][enlarged_cum_sum[i - 1]:enlarged_cum_sum[i]]))
            # print(enlarged_cum_sum[i - 1], enlarged_cum_sum[i])
        dic[res_id[i]] += 1

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


F140763.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


F140518.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

F140850.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


F140304.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


F140112.WAV


## Male

In [6]:
output_root_path = "/home/oscar/Documents/Workspace/11785/project/penstate_data/extract_phoneme/male/"

In [7]:
counter = 332
for voice_file in male_list[332:]:
    print(counter) # counter number of file handled
    counter+=1;
    
    print(voice_file)
    voice_absolute_path = male_path + "/" + voice_file
    people_id = voice_file.split(".")[0]
    # print(voice_absolute_path)
    voice, sample_rate = torchaudio.load(voice_absolute_path)
    input_values = processor(voice[0][::2], return_tensors="pt").input_values
    with torch.no_grad():
        logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    # transcription = processor.batch_decode(predicted_ids)
    
    ratio = len(voice[0]) / len(predicted_ids[0])
    
    res_len, res_id = get_phoneme_len(predicted_ids[0])
    
    cum_sum = np.cumsum(res_len)
    enlarged_cum_sum = (cum_sum * ratio).astype(int)
    
    dic = defaultdict(int)
    
    for i in range(len(res_id)):
        if not os.path.exists(output_root_path + str(res_id[i])):
            os.mkdir(output_root_path + str(res_id[i]))
        curr_phoneme_output_path = output_root_path + str(res_id[i]) + "/" + people_id + '_' + str(res_id[i]) + '_' + str(dic[res_id[i]]) + '.npy'
        if i == 0:
            np.save(curr_phoneme_output_path, np.array(voice[0][max(enlarged_cum_sum[i] - 5000, 0):enlarged_cum_sum[i]]))
            # print(max(enlarged_cum_sum[i] - 20, 0), enlarged_cum_sum[i])
        elif i == len(res_id)-1:
            np.save(curr_phoneme_output_path, np.array(voice[0][enlarged_cum_sum[i - 1]:min(enlarged_cum_sum[i], enlarged_cum_sum[i - 1] + 20)]))
            # print(enlarged_cum_sum[i - 1], min(enlarged_cum_sum[i], enlarged_cum_sum[i - 1] + 5000))
        else:
            np.save(curr_phoneme_output_path, np.array(voice[0][enlarged_cum_sum[i - 1]:enlarged_cum_sum[i]]))
            # print(enlarged_cum_sum[i - 1], enlarged_cum_sum[i])
        dic[res_id[i]] += 1

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


332
M140067.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


333
M140109.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


...

411
M140915.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


412
M140530.WAV


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


413
M140661.WAV


In [8]:
print("complete")

complete
